<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Treinamento_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Configurações Iniciais 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import string, re
import itertools
import nltk
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.callbacks import EarlyStopping
py.init_notebook_mode(connected=True)
%matplotlib inline
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

## Lendo a Base

In [ ]:
# Base1 Com StopWords
PATH_BASE1 = "/content/drive/MyDrive/TCC/dados/processadas/Base1_classificada.csv"

# Base2 Sem StopWords
PATH_BASE2 = "/content/drive/MyDrive/TCC/dados/processadas/Base2_classificada.csv"

# Base1 Com StopWords
PATH_BASE1_JUNTO_COM_DA_LEILA = "/content/drive/MyDrive/TCC/dados/processadas/Base1JuntaHateENotHateDaLeila.csv"

# Base2 Sem StopWords
PATH_BASE2_LEILA_LIMPISSIMA = "/content/drive/MyDrive/TCC/dados/processadas/Base2_maior4_menor25_limpissima.csv"

In [ ]:
#trainDF = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA, index_col=0)
trainDF = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA)

In [ ]:
# Agrupa dados por label
trainDF.groupby('label').count()

,text
label,
hate,2443
not_hate,19282


In [ ]:
categorias_desejadas = ['hate', 'not_hate'] # !!!
df_filtrado = trainDF[trainDF.label.isin(categorias_desejadas)]
df_filtrado.shape

(21725, 2)

In [ ]:
df_filtrado

,label,text
0,not_hate,geral foi no jogo do foz
1,not_hate,pois é não é gente é isso ai pessoal
2,not_hate,ontem eu estava tão mal que escrevi tudo o que...
3,not_hate,eu devo ter um coração de pedra não sou de me ...
4,not_hate,camimq vivobr pois é estou tentando entender
...,...,...
21720,hate,eduguim depois falam que nordestino não sabe ...
21721,hate,e dizem por aí que o nordestino não sabe votar
21722,hate,ain que nordestino não sabe votar e blá blá blá
21723,hate,imagine essa turma da extrema direita soltando...


In [ ]:
def balanceia_dados(df, verbose=0):
    # Esta funcao recebe um dataframe original e retorna um dataset
    # contendo um numero igual de noticias das categorias
    # por simplicidade, vou supor que são somente duas categorias.
    global categorias_desejadas
    df_balanceado = df.copy()
    
    # cria um dataframe para cada classe. Assim podemos remover o excesso
    df_F = df_balanceado.loc[df_balanceado.label == categorias_desejadas[0]]
    df_T = df_balanceado.loc[df_balanceado.label == categorias_desejadas[1]]

    num_amostras_F, num_amostras_T = df_F.shape[0], df_T.shape[0]
    num_amostras_a_remover = abs(num_amostras_F - num_amostras_T)
    
    # remove amostras da classe majoritaria
    if num_amostras_F > num_amostras_T:
        #print(df_F[:num_amostras_a_remover)
        df_F = df_F[num_amostras_a_remover:]
        
    elif num_amostras_T > num_amostras_F:
        #
        df_T = df_T[num_amostras_a_remover:]

    return pd.concat([df_F, df_T])

In [ ]:
# Aplica função de balancear os dados
dataBC = balanceia_dados(df_filtrado)

# DATA BALANCEADO
trainDF = dataBC
trainDF.reset_index(drop=True, inplace=True) # reseta index
trainDF.shape

(4886, 2)

In [ ]:
# convert output  hate =1 not_hate =0
trainDF.label = [1 if each == "hate" else 0 for each in trainDF.label] 

In [ ]:
trainDF.groupby('label').count()

,text
label,
0,2443
1,2443


In [ ]:
# Pega a média de caracteres dos tweets de toda a base
average_len = int(trainDF['text'].apply(lambda x: len(str(x))).mean())
average_len

62

In [ ]:
quantidade_palavras_unicas = len( list ( trainDF['text'].str.split(' ', expand=True).stack().unique() ) )
quantidade_palavras_unicas

9144

# word embeddings

In [ ]:
# WORD EMBEDDING cbow s50 filtrado
PATH_WORD_EMBEDDING_CBOW_50FILTRADO = "/content/drive/MyDrive/TCC/dados/word_embeddings/cbow_s50_filtrado.txt"

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
model = KeyedVectors.load_word2vec_format(PATH_WORD_EMBEDDING_CBOW_50FILTRADO)

In [ ]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open(PATH_WORD_EMBEDDING_CBOW_50FILTRADO)):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [ ]:
# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index


In [ ]:
tam= len(word_index)
tam

9147

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.3, random_state=2)

In [ ]:
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x)) # tiramos o parâmetro maxlen, então usa o tam do maior tweet
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x))

In [ ]:
# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# CNN model

In [ ]:
from sklearn import metrics
from keras import layers, models, optimizers

def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(classifier.summary())
    classifier.fit(feature_vector_train, label, epochs=3)
    return classifier

def create_rnn_lstm(embedding_matrix):
  
  model_input = layers.Input(shape=(50, ),name="input",dtype='int32')
  embedding = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix])(model_input)
  embedding = layers.SpatialDropout1D(0.3)(embedding)
  lstm = layers.LSTM(1000, dropout =0.1, name="LSTM")(embedding)
  output = layers.Dense(1, activation='sigmoid',name='sigmoid')(lstm)
  model = models.Model(inputs=model_input,outputs=output)
  return model

classifier = create_rnn_lstm(embedding_matrix)
trainedModel = train_model(classifier, train_seq_x, train_y, valid_seq_x)
# predict the labels on validation dataset
predictions = trainedModel.predict(valid_seq_x)
predictions = predictions.round()

print("RNN-LSTM, Word Embeddings",  metrics.accuracy_score(valid_y,predictions))
print(predictions)

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label, epochs=10) #adicionamos epoca
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [ ]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((24, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=True)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(64, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    #sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='sgd', loss='mean_squared_error', metrics = ['acc', 'mae'])
  
    return model

In [ ]:
classifier = create_cnn()
accuracyCNN = train_model(classifier,train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings - accuracy",  accuracyCNN)

Epoch 1/10
107/107 [==============================] - 1s 6ms/step - loss: 0.2845 - acc: 0.5015 - mae: 0.4946
Epoch 2/10
107/107 [==============================] - 1s 7ms/step - loss: 0.2700 - acc: 0.4895 - mae: 0.5030
Epoch 3/10
107/107 [==============================] - 1s 6ms/step - loss: 0.2590 - acc: 0.5003 - mae: 0.4995
Epoch 4/10
107/107 [==============================] - 1s 6ms/step - loss: 0.2562 - acc: 0.5015 - mae: 0.4988
Epoch 5/10
107/107 [==============================] - 1s 7ms/step - loss: 0.2553 - acc: 0.5067 - mae: 0.5000
Epoch 6/10
107/107 [==============================] - 1s 6ms/step - loss: 0.2519 - acc: 0.5190 - mae: 0.4969
Epoch 7/10
107/107 [==============================] - 1s 6ms/step - loss: 0.2510 - acc: 0.5278 - mae: 0.4970
Epoch 8/10
107/107 [==============================] - 1s 6ms/step - loss: 0.2522 - acc: 0.5164 - mae: 0.4986
Epoch 9/10
107/107 [==============================] - 1s 6ms/step - loss: 0.2527 - acc: 0.5003 - mae: 0.4997
Epoch 10/10
107/107